In [12]:
import json

import numpy as np
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.14.0


# Load the MNIST dataset

In [58]:
import json

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

def flatten(two_d_array):
    """
    Flattens a two-dimensional array into a one dimensional array
    Helpful for exporting the 784 pixels used in the MNIST dataset
    so that we can test the Rust model.
    :param two_d_array: [[int]] 
    :return: 
    """
    return [int(item) for sublist in two_d_array for item in sublist]

# Export 1 example for each digit
found = []
while len(found) < 10:
    for i in range(len(y_train)):
        if y_train[i] not in found:
            found.append(y_train[i])
            example = x_train[i]
            flattened_example = flatten(example)
            with open("examples/example{}.json".format(y_train[i]), "w") as f:
                f.write(json.dumps(flattened_example))
            break

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build the Model

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1)

2023-10-06 16:47:33.762597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 13s 6ms/step - loss: 0.4081 - accuracy: 0.8822


In [5]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 2s - loss: 0.2993 - accuracy: 0.9170 - 2s/epoch - 6ms/step


[0.29934269189834595, 0.9169999957084656]

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Test the Model

In [47]:
# This is the number seven pulled from the first test case in the MNIST dataset
example = x_test[:1]

# I'm going to get a prediction which is going to give me an array of probabilities, where each
# element in the array corresponds to the probability that the image is that number
prediction = model.predict(example)

# And I'll print the index of the highest probability
np.argmax(prediction)

1/1 [==============================] - 0s 56ms/step


7

# Export the Model

In [76]:
# Let's figure out what weights we need to export. 
# The dense layer has weights and biases, so we'll extract both of those
dense1 = model.layers[1].get_weights()
dense_weights1 = dense1[0]
dense_biases1 = dense1[1]

print("Dense 1 Weights Shape:", dense_weights1.shape)
print("Dense 1 Biases Shape:", dense_biases1.shape)

# Dropout is not a layer that is applied during inference, so it doesn't have to be exported
# Let's export the weights of the final dense layer
dense2 = model.layers[3].get_weights()
dense_weights2 = dense2[0]
dense_biases2 = dense2[1]

print("Dense 2 Weights Shape:", dense_weights2.shape)
print("Dense 2 Biases Shape:", dense_biases2.shape)


Dense 1 Weights Shape: (784, 128)
Dense 1 Biases Shape: (128,)
Dense 2 Weights Shape: (128, 10)
Dense 2 Biases Shape: (10,)


In [77]:
model.layers

In [71]:
from safetensors.tensorflow import save_file

save_file({
    "d1_w": tf.convert_to_tensor(dense_weights1),
    "d1_b": tf.convert_to_tensor(dense_biases1),
    "d2_w": tf.convert_to_tensor(dense_weights2),
    "d2_b": tf.convert_to_tensor(dense_biases2),
}, "model.safetensors")